In [0]:
#!pip install git+https://github.com/haven-jeon/PyKoSpacing.git
!pip install git+https://github.com/monologg/KoELECTRA.git
!pip install attrdict

  Cloning https://github.com/monologg/KoELECTRA.git to /tmp/pip-req-build-xnugeh6_
  Running command git clone -q https://github.com/monologg/KoELECTRA.git /tmp/pip-req-build-xnugeh6_
  Resolved https://github.com/monologg/KoELECTRA.git to commit 024fbdd600e653b6e4bdfc64ceec84181b5ce6c4
ERROR: File "setup.py" not found for legacy project git+https://github.com/monologg/KoELECTRA.git.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-70864872-2a3a-4af0-a1e4-0856fb99eda3/bin/python -m pip install --upgrade pip' command.
  Using cached attrdict-2.0.1-py2.py3-none-any.whl (9.9 kB)
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-70864872-2a3a-4af0-a1e4-0856fb99eda3/bin/python -m pip install --upgrade pip' command.


In [0]:
import argparse
import json
import logging
import os
import glob
import re
#from pykospacing import Spacing
import numpy as np
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.nn import CrossEntropyLoss
from fastprogress.fastprogress import master_bar, progress_bar
import pandas as pd 
from attrdict import AttrDict
from transformers import AutoTokenizer, ElectraTokenizer

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-3722261225414716>:14
     12 from fastprogress.fastprogress import master_bar, progress_bar
     13 import pandas as pd 
---> 14 from attrdict import AttrDict
     15 from transformers import AutoTokenizer, ElectraTokenizer

File /databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py:171, in _create_import_patch.<locals>.import_patch(name, globals, locals, fromlist, level)
    166 thread_local._nest_level += 1
    168 try:
    169     # Import the desired module. If you’re seeing this while debugging a failed import,
    170     # look at preceding stack frames for relevant error information.
--> 171     original_result = python_builtin_import(name, globals, locals, fromlist, level)
    173     is_root_import = thread_local._nest_level == 1
    174     # `level` represents the number of leading 

In [0]:
#data = pd.DataFrame(columns=['text', 'label'])

### Create and Label the Train Data
#### {해당없음: 'o', 입원일자: 'date_h', 퇴원일자: 'date_h', 수술일자: 'date_s', 진단일자: 'date_d'}

In [0]:
data = pd.read_csv('/Workspace/Shared/MYJ/03.OCR/KoELECTRA-master/finetune/data/naver-ner/train_v2.csv', sep='\t', header=None)
data.columns = ['text', 'label']

In [0]:
#data
data.tail()

text 
 label 
 
 
 
 
 104 
 양측 대복재 정맥 01 초 이상 지속되는 역류 확인되어 2023.02.23 시아노아... 
 O O O O O O O O O DATE_S O O SURG SURG O O O 
 
 
 105 
 상기 환자는 상기 병명 2023.02.06 부터 2023.02.10 입원 치료하였고... 
 O O O O DATE_H O DATE_R O O O O O O O O O O 
 
 
 106 
 상기인 2023.08.22 본원서 시행한 대장내시경상 용종이 관찰되어 , 올가미 용... 
 O DATE_S O O O O O O SURG SURG O O O O O O O O... 
 
 
 107 
 상기 질환 내원하여 2023.02.27 우측 대복재정맥 0.05 초 이상의 역류소견... 
 O O O DATE_S O O O O O O O O SURG SURG O O O O... 
 
 
 108 
 상기환자는 2022.03.21 본원 내원하여 상기병명 진단 후 2022.03.21 ... 
 O DATE_V O O O O O DATE_S O O O O DATE_S O O O...

In [0]:
re.sub(r'(\d{1,4})\s+\.\s+(\d{1,4})\s+\.\s+(\d{1,4})', r'\1.\2.\3', '2023 .8 .1')

Out[1055]: '2023 .8 .1'

In [0]:
'''
regex = r'[0-9]'

date_formats = [r'\d{4}년\d{1,2}월\d{1,2}일',
                r'\d{1,2}월\d{1,2}일',
                r'\d{1,4}/\d{1,2}/\d{1,2}',
                r'\d{1,2}/\d{1,2}',
                r'\d{1,4}.\d{1,2}.\d{1,2}',
]

current_year = None 
current_month = None
month_day = ['월', '일']
def spacing(text, current_year = None, current_month=None):
    final_text = []
    text = text.replace('  ', ' ').replace('- ', '-').replace(' -', '-').replace('/ ', '/').replace(' /', '/').replace('년 ', '년').replace(' 년', '년').replace('월 ', '월').replace(' 월', '월').replace('일 ', '일').replace(' 일', '일')
    text = re.sub('(\d+(\.\d+)?)', r' \1 ', text).strip() # 띄어쓰기
    text = re.sub('\s*([0-9])\s', r'\1', text) #필요없는 띄어쓰기 제거

    
    #text = re.sub(r'(\d{1,4})\년(\d{1,4})\월\s+(\d{1,4})\일' , r'\1.\2.\3', text) # . 사이 띄어쓰기 제거
    #text = re.sub(r'(\d{1,4})\년(\d{1,4})\월\s+(\d{1,4})\일' , r'\1.\2.\3', text) # . 사이 띄어쓰기 제거
    text = re.sub(r'(\d{1,4})\년(\d{1,4})\월(\d{1,4})\일' , r'\1.\2.\3', text) # . 사이 띄어쓰기 제거
    #text = re.sub(r'(\d{1,4})\년\s+(\d{1,4})\월\s+(\d{1,4})\일' , r'\1.\2.\3', text) # . 사이 띄어쓰기 제거
    text = re.sub(r'(\d{1,4})\.\s+(\d{1,4})\.\s+(\d{1,4})', r'\1.\2.\3', text) # . 사이 띄어쓰기 제거
    text = re.sub(r'(\d{1,4})\/\s+(\d{1,4})\/\s+(\d{1,4})', r'\1.\2.\3', text) # / 사이 띄어쓰기 제거
    text = re.sub(r'(\d{1,4})\/(\d{1,4})\/\s+(\d{1,4})', r'\1.\2.\3', text) # / 사이 띄어쓰기 제거
    text = re.sub(r'(\d{1,4})\/\s+(\d{1,4})\/(\d{1,4})', r'\1.\2.\3', text) # / 사이 띄어쓰기 제거
    text = re.sub(r'(\d{1,4})\-(\d{1,4})\-(\d{1,4})', r'\1.\2.\3', text) # - 사이 띄어쓰기 제거
    text = re.sub(r'(\d{1,4})\/(\d{1,4})\/(\d{1,4})', r'\1.\2.\3', text) # / 형태를 . 형태로 바꾸기
    text = re.sub(r'(\d{1,4})\.\s+(\d{1,4})\.\s+(\d{1,4})', r'\1.\2.\3', text) # . 사이 띄어쓰기 제거
    text = re.sub(r'(\d{1,4})\.\s+(\d{1,4})', r'\1.\2', text) # . 사이 띄어쓰기 제거
    text = re.sub(r'(\d{1,4})\/\s+(\d{1,4})', r'\1/\2', text) # / 사이 띄어쓰기 제거
    text = text.replace('~', ' ~ ')
    text = text.replace(',', ' , ')
    text = text.replace('(', ' ( ').replace(')', ' ) ')
    text = text.replace('에','').replace('를','').replace('에서','').replace('까지','').replace('부터',' 부터') #불용어 제거
    text = text.replace('년', '년 ').replace('월', '월 ').replace('일', '일 ')
    text = text.replace('  ', ' ')

    for word in text.split():
        if  ('년' in word) & (len(re.sub(r'[^0-9]', '', word))==2):
            word = '20' + re.sub(r'[^0-9]', '', word) + '년' #숫자 + 년  
        elif any (date_word in word for date_word in month_day) & (len(re.sub(r'[^0-9]', '', word))==1):
            str_date = re.sub(r'[0-9]', '', word)
            word = re.sub(r'[^0-9]', '', word).zfill(2) + str_date
        elif (len(re.sub(r'[^0-9]', '', word))==1): 
            word = re.sub(r'[^0-9]', '', word).zfill(2)

        ### yyyy/mm/dd 형태 전처리 ###
        '''
        if ('/' in word):
            split_date_final = []
            only_int = []
            split_date = word.split('/')
            idx = 0
            for date in split_date:
                date =  re.sub(r'[^0-9]', '', date).zfill(2)
                only_int += [date]
                if idx < len(split_date) - 1:
                    split_date_final += [date + '/']
                else: 
                    split_date_final += [date]
                idx += 1 
            if len(''.join(only_int)) == 6:
                word = '20' + ''.join(split_date_final)
            else: word = ''.join(split_date_final)
            if current_year == None:
                current_year = re.sub(r'[^0-9]', '', word)[:4] + '년'
            if current_month == None:
                current_month = re.sub(r'[^0-9]', '', word)[4:6]+'월'
                '''
        ### yyyy.mm.dd 형태 전처리 ###
        if ('/' in word) & (any (re.match(date_format, word) for date_format in date_formats)) :
            split_date_final = []
            only_int = []
            split_date = word.split('/')
            idx = 0
            for date in split_date:
                date =  re.sub(r'[^0-9]', '', date).zfill(2)
                only_int += [date]
                if idx < len(split_date) - 1:
                    split_date_final += [date + '/']
                else: 
                    split_date_final += [date]
                idx += 1 
            if len(''.join(only_int)) == 6:
                word = '20' + ''.join(split_date_final)
            else: word = ''.join(split_date_final)
            only_int = ''.join(only_int)
            if current_year == None:
                current_year = re.sub(r'[^0-9]', '', word)[:4] + '년'
            if current_month == None:
                current_month = re.sub(r'[^0-9]', '', word)[4:6] + '월'
        ### yyyy.mm.dd 형태 전처리 ###
        if ('' in word) & (any (re.match(date_format, word) for date_format in date_formats)) :
            split_date_final = []
            only_int = []
            split_date = word.split('.')
            idx = 0
            for date in split_date:
                date =  re.sub(r'[^0-9]', '', date).zfill(2)
                only_int += [date]
                if idx < len(split_date) - 1:
                    split_date_final += [date + '.']
                else: 
                    split_date_final += [date]
                idx += 1 
            if len(''.join(only_int)) == 6:
                word = '20' + ''.join(split_date_final)
            else: word = ''.join(split_date_final)
            only_int = ''.join(only_int)
            if current_year == None:
                current_year = re.sub(r'[^0-9]', '', word)[:4] + '년'
            if current_month == None:
                current_month = re.sub(r'[^0-9]', '', word)[4:6] + '월'
        #동년 동월 처리 
        if (current_year == None) & ('년' in word): 
            current_year = word
        if (current_month == None) & ('월' in word): 
            current_month = word
        if (current_year != None) & ('동년' in word): 
            word = current_year
        if (current_month != None) & ('동월' in word): 
            word = current_month
        final_text += [word]
    
    final_text = ' '.join(final_text)
    return final_text
    '''

In [0]:
import datetime as dt
date_formats = [r'\d{4}년\d{1,2}월\d{1,2}일',
                r'\d{1,2}월\d{1,2}일',
                r'\d{1,4}/\d{1,2}/\d{1,2}',
                r'\d{1,2}/\d{1,2}',
                r'\d{1,4}.\d{1,2}.\d{1,2}',
                r'\d{1}.\d{1}'
]
current_year = None 
current_month = None
month_day = ['월', '일']
def spacing(text, current_year = None, current_month=None):
    final_text = []
    text = re.sub('(\d+(\.\d+)?)', r' \1 ', text).strip() # 띄어쓰기
    text = ' '.join(text.split())
    text = re.sub(r'(\d{1,4})\s+\.\s+(\d{1,4})\s+\.\s+(\d{1,4})', r'\1.\2.\3', text) # . 사이 띄어쓰기 제거
    text = re.sub(r'(\d{1,4})\.(\d{1,4})\s+\.\s+(\d{1,4})', r'\1.\2.\3', text) # . 사이 띄어쓰기 제거 (뒤에만 띄어쓰기 있을 경우)
    text = re.sub(r'(\d{1,4})\s+\/\s+(\d{1,4})\s+\/\s+(\d{1,4})', r'\1.\2.\3', text) # / 사이 띄어쓰기 제거
    text = re.sub(r'(\d{1,4})\s+\-\s+(\d{1,4})\s+\-\s+(\d{1,4})', r'\1.\2.\3', text) # - 사이 띄어쓰기 제거
    text = re.sub(r'(\d{1,4})\s+\년\s+(\d{1,4})\s+\월\s+(\d{1,4})\s+\일', r'\1.\2.\3', text) # 년월일 사이 띄어쓰기 제거
    text = re.sub(r'(\d{1,4})\s+\월\s+(\d{1,4})\s+\일', r'\1.\2', text) # 월 일 만 있는 부분 사이 띄어쓰기 제거
    text = text.replace('~', ' ~ ')
    text = text.replace(',', ' , ')
    text = text.replace('(', ' ( ').replace(')', ' ) ')
    text = text.replace('에서','').replace('으로','').replace('에','').replace('을','').replace('로','').replace('를','').replace('까지','').replace('부터',' 부터').replace('"', '').replace("'", '').replace('동년동월', '동년 동월') #불용어 제거 및 띄어쓰기
    #text = text.replace('년', '년 ').replace('월', '월 ').replace('일', '일 ')
    #text = text.replace('  ', ' ')
    for word in text.split():
        if  ('년' in word) & (len(re.sub(r'[^0-9]', '', word))==2):
            word = '20' + re.sub(r'[^0-9]', '', word) + '년' #숫자 + 년  
        elif any (date_word in word for date_word in month_day) & (len(re.sub(r'[^0-9]', '', word))==1):
            str_date = re.sub(r'[0-9]', '', word)
            word = re.sub(r'[^0-9]', '', word).zfill(2) + str_date
        elif (len(re.sub(r'[^0-9]', '', word))==1): 
            word = re.sub(r'[^0-9]', '', word).zfill(2)

        ### yyyy.mm.dd 형태 전처리 ###
        if ('.' in word) & (any (re.match(date_format, word) for date_format in date_formats)) :
            split_date_final = []
            only_int = []
            split_date = word.split('.')
            idx = 0
            for date in split_date:
                if date != '0':   # 앞에 0일 경우에는 시간이 아님
                    date =  re.sub(r'[^0-9]', '', date).zfill(2)
                only_int += [date]
                if idx < len(split_date) - 1:
                    split_date_final += [date + '.']
                else: 
                    split_date_final += [date]
                idx += 1 
            if len(''.join(only_int)) == 6:
                word = '20' + ''.join(split_date_final)
                current_year = word[:4]
                current_month = only_int[4:6]
                current_day = only_int[6:8]
            else: word = ''.join(split_date_final)
            only_int = ''.join(only_int)
            if len(only_int)== 8:
                current_year = only_int[:4]
                current_month = only_int[4:6]
                current_day = only_int[6:8]
            try:
                if len(only_int)==4:
                    word = current_year + '.' + word
            except:
                pass
            #if current_year == None:
               #current_year = re.sub(r'[^0-9]', '', word)[:4] + ' .'
            #if current_month == None:
                #current_month = re.sub(r'[^0-9]', '', word)[4:6] + ' .'
        #동년 동월 처리 
        if (current_year == None) & ('년' in word): 
            current_year = word
        if (current_month == None) & ('월' in word): 
            current_month = word
        ############### 2023-11-13 일 한글에 대한 추가 전처리 ###############
        if ('다음날' in word):
            word = current_year + '.' + current_month +  '.' + current_day
            word = dt.datetime.strptime(word, '%Y.%m.%d') + dt.timedelta(days=1)
            word = dt.datetime.strftime(word, '%Y.%m.%d')
        #####################################################################
        final_text += [word]
        #print(only_int)

    final_text = ' '.join(final_text)
    final_text = re.sub(r'(\d{1,4})\s+\.\s+(\d{1,4})\s+\.\s+(\d{1,4})', r'\1.\2.\3', final_text) # . 사이 띄어쓰기 제거 (동년 동월에 대한 띄어쓰기 제거)
    final_text = re.sub(r'(\d{1,4})\s+\.\s+(\d{1,4})\.(\d{1,4})', r'\1.\2.\3', final_text) # . 사이 띄어쓰기 제거 (동월에 대한 띄어쓰기 제거)
    return final_text

In [0]:
sentence_processed = []
sentence = '상기환자는 2022년 3월 21일 본원 내원하여 상기병명으로 진단 후 2022년 3월 21일 (좌안), 3월 22일 (우안) 백내장초음파 유화술 및 후방 다초점 인공수정체 삽입술을 본원에서 시행받았음. 단초점렌즈 사용 시 조절장애의 치료효과를 기대하기 어려워 다초점 인공수정체를 사용하였음. 상기 수술 및 치료재료대는 백내장 치료목적이며, 향후 경과관찰 요함'
sentence_processed = spacing(sentence)
sentence_processed

Out[69]: '상기환자는 2022.03.21 본원 내원하여 상기병명 진단 후 2022.03.21 ( 좌안 ) , 2022.03.22 ( 우안 ) 백내장초음파 유화술 및 후방 다초점 인공수정체 삽입술 본원 시행받았음. 단초점렌즈 사용 시 조절장애의 치료효과 기대하기 어려워 다초점 인공수정체 사용하였음. 상기 수술 및 치료재료대는 백내장 치료목적이며 , 향후 경과관찰 요함'

In [0]:
words = sentence_processed.split()
temp = ['O'] *  len(words) 
for idx, word in enumerate(words):
    print(idx, word)

0 상기환자는
1 2022.03.21
2 본원
3 내원하여
4 상기병명
5 진단
6 후
7 2022.03.21
8 (
9 좌안
10 )
11 ,
12 2022.03.22
13 (
14 우안
15 )
16 백내장초음파
17 유화술
18 및
19 후방
20 다초점
21 인공수정체
22 삽입술
23 본원
24 시행받았음.
25 단초점렌즈
26 사용
27 시
28 조절장애의
29 치료효과
30 기대하기
31 어려워
32 다초점
33 인공수정체
34 사용하였음.
35 상기
36 수술
37 및
38 치료재료대는
39 백내장
40 치료목적이며
41 ,
42 향후
43 경과관찰
44 요함


In [0]:
temp[1] = 'DATE_V'
temp[7] = 'DATE_S'
temp[12] = 'DATE_S'
temp[16] = 'SURG'
temp[17] = 'SURG'
temp[19] = 'SURG'
temp[20] = 'SURG'
temp[21] = 'SURG'
temp[22] = 'SURG'

In [0]:
temp_join = ' '.join(temp)

In [0]:
data = data.append(pd.Series([sentence_processed, temp_join], index = data.columns ), 
           ignore_index=True)

<command-3722261225414527>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(pd.Series([sentence_processed, temp_join], index = data.columns ),


In [0]:
data[data['label'].str.contains('HOSP')]
#print(data.iloc[23,:]['label'], len(data.iloc[23,:]['label'].split()) )
#data.tail(12)
#data = data.drop([109]).reset_index(drop=True)

text 
 label

In [0]:
#data['label'] =  data['label'].str.replace('HOSP', 'O')

In [0]:
data.to_csv('/Workspace/Shared/MYJ/03.OCR/KoELECTRA-master/finetune/data/naver-ner/train_v2.csv', sep='\t', index=False, header=False)

### Create and Label the Test Data

In [0]:
#test_data = pd.DataFrame(columns=['text', 'label'])

In [0]:
test_data = pd.read_csv('/Workspace/Shared/MYJ/03.OCR/KoELECTRA-master/finetune/data/naver-ner/test.csv', sep='\t', header=None)
test_data.columns = ['text', 'label']

In [0]:
sentence = [spacing('상기 환자 요통 및 하지 방사통 주증상으로 내원한 환자로 본원 시행검사상 상기병명 진단되어 2023년 4월 27일 "경막외유착박리술 및 신경성형술" 시행한 환자로 물리치료 위해 의뢰드립니다. 감사합니다')]
sentence

Out[169]: ['상기 환자 요통 및 하지 방사통 주증상 내원한 환자 본원 시행검사상 상기병명 진단되어 2023.04.27 경막외유착박리술 및 신경성형술 시행한 환자 물리치료 위해 의뢰드립니다. 감사합니다']

In [0]:
words = sentence[0].split()
temp = ['O'] *  len(words) 
for idx, word in enumerate(words):
    print(idx, word)

0 상기
1 환자
2 요통
3 및
4 하지
5 방사통
6 주증상
7 내원한
8 환자
9 본원
10 시행검사상
11 상기병명
12 진단되어
13 2023.04.27
14 경막외유착박리술
15 및
16 신경성형술
17 시행한
18 환자
19 물리치료
20 위해
21 의뢰드립니다.
22 감사합니다


In [0]:
temp[13] = 'DATE_S'
temp[14] = 'SURG'
temp[16] = 'SURG'

In [0]:
temp_join = ' '.join(temp)

In [0]:
test_data = test_data.append(pd.Series([sentence[0], temp_join], index = test_data.columns ), 
           ignore_index=True)

<command-4090120528936839>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_data = test_data.append(pd.Series([sentence[0], temp_join], index = test_data.columns ),


In [0]:
test_data['label'] =  test_data['label'].str.replace('DISE', 'O')

In [0]:
test_data.to_csv('/Workspace/Shared/MYJ/03.OCR/KoELECTRA-master/finetune/data/naver-ner/test.csv', sep='\t', index=False, header=False)

In [0]:
#test_data[test_data['label'].str.contains('DATSURGE_R')]

text 
 label

### FINETUNE MODEL 

In [0]:
cd /Workspace/Shared/MYJ/03.OCR/KoELECTRA-master/finetune

/Workspace/Shared/MYJ/03.OCR/KoELECTRA-master/finetune


In [0]:
!python3 run_ner.py --task naver-ner --config_file koelectra-small-v3.json #모형 결과값은 /Shared/MYJ/03.OCR/KoELECTRA-master/finetune/data  저장 

2023-11-13 05:20:12.879221: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Some weights of the model checkpoint at monologg/koelectra-small-v3-discriminator were not used when initializing ElectraForTokenClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForTokenClassification were 

### Model Load and Predict

In [0]:
cd /Workspace/Shared/MYJ/03.OCR

/Workspace/Shared/MYJ/03.OCR


In [0]:
from transformers import ElectraTokenizer, ElectraForTokenClassification
from ner_pipeline_v3 import ner_scoring
from pprint import pprint

tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-small-v3-discriminator")
model = ElectraForTokenClassification.from_pretrained("/Workspace/Shared/MYJ/03.OCR/KoELECTRA-master/koelectra-small-naver-ner-ckpt/checkpoint-480")

ner = ner_scoring(model=model,
                  tokenizer=tokenizer,
                  ignore_labels=[], #['O''RELE', 'HOSP', 'FROM_TO'],
                  ignore_special_tokens=True)

In [0]:
ner.extraction_to_print(spacing('난소의 양성 신생물, 비염증성 장애로 인한 복강경하 우측 난소종양적출술 및 부난관낭종 제거술을 시행 하였으며, 향후 일정기간 안정가료 및 추적진료 요망됩니다. 수술일자: 2023-09-07 입원일자: 2023-09-06 ~ 2023-09-10'))

입원일 : ['2023.09.06']
퇴원일 : ['2023.09.10']
수술일 : ['2023.09.07']
수술명 : ['복강경하', '난소종양적출술', '제거술']


In [0]:
ner.extraction_to_print(spacing("2023년 3월 19일 절제술 시행함. 2023년 3월 20일 ~ 2023년 3월 27일 까지 입원했음 "))

입원일 : ['2023.03.20']
퇴원일 : ['2023.03.27']
수술일 : ['2023.03.19']
수술명 : ['절제술']


In [0]:
ner.extraction_to_print(spacing(" 2020년 1월 1일 입원하여 2월 3일에 퇴원 입원중 디스크 진단으로 1월 31일에 척추 절제술 함"))

입원일 : ['2020.01.01']
퇴원일 : ['2020.02.03']
수술일 : ['2020.01.31']
수술명 : ['척추 절제술']


In [0]:
ner.extraction_to_print(spacing('상기환자 상기상병에 대해 2023년 6월 12일 본원 정형외과 입원하여 동년 6월 13일 척골 신경 전방 전위술, 신경 이식술 시행 후 동년 6월 17일 퇴원함. 추후 외래 추시 필요할 것으로 사료됨. 단, 상기 상병은 정형외과 수부 영역에 한하며 추후 미발증, 속발증, 병발증 발생 시 진단명 변경될 수 있음'))

입원일 : ['2023.06.12']
퇴원일 : ['2023.06.17']
수술일 : ['2023.06.13']
수술명 : ['척골 신경 전방 전위술', '신경 이식술']


In [0]:
ner.extraction_to_print(spacing('상병명에 대하여 2023년 4월 26일 본원 신경외과에 입원하여 2023년 4월 27일 개두술 및 뇌동맥류 클립결찰술 시행 후 2023년 5월 6일 퇴원한 환자로 추후 정기적인 추적관찰을 요함'))

입원일 : ['2023.04.26']
퇴원일 : ['2023.05.06']
수술일 : ['2023.04.27']
수술명 : ['개두술', '뇌동맥류 클립결찰술']


In [0]:
ner.extraction_to_print(spacing('상기 환자는 상기 병명으로 2021년 12월 31일 ~ 2022년 1월 12일 까지 입원하였음.  2022년 1월 10일 시행함'))

입원일 : ['2021.12.31']
퇴원일 : ['2022.01.12']
수술일 : ['2022.01.10']
수술명 : []


In [0]:
ner.extraction_to_print(spacing(" 2023-06-20 부터 2023-07-04 까지 15 일간 수술 및 치료목적으로 비급여 항목 사용하였음 다한실 부족 및 감염예방을 위하여 상급병실 이용하였음 구호 영양공급이 불충분하여 수액요법 치료를 실시함 "))

입원일 : ['2023.06.20']
퇴원일 : ['2023.07.04']
수술일 : []
수술명 : ['수액요법 치료']


In [0]:
ner.extraction_to_print(spacing('상기PT는 2023년 6월 21일 내원하여 검사 후 2011년 11월 21일 발치된 27과 2023년 1월 26일 발치된 #26 #25 부위에 치조골이식과 동시에 임플란트 식립함'))

입원일 : []
퇴원일 : []
수술일 : ['2023.06.21']
수술명 : ['치조골이식과', '임플란트']


In [0]:
ner.extraction_to_print(spacing('위 환자는 2023년 4월 10일 좌측 유방 부분 절제술 및 동측액와 감시림프절 생검술 시행 받았고 2023년 04월 09일부터 2023년 04월 12일까지 입원치료하였으며 조직검사 결과, 왼쪽 유방 진단되었음'))

입원일 : ['2023.04.09']
퇴원일 : ['2023.04.12']
수술일 : ['2023.04.10']
수술명 : ['유방 부분 절제술', '감시림프절 생검술']


In [0]:
(spacing('상기 환자 진단 하 2023.4.12 입원하여 동년동월 13일 복강경하 좌측난소 낭종절제술, 자궁근종절제술 시행받고 동년동월 16일퇴원한 환자임'))

Out[30]: '상기 환자 진단 하 2023.04.12 입원하여 동년 동월 13 일 복강경하 좌측난소 낭종절제술 , 자궁근종절제술 시행받고 동년 동월 16 일퇴원한 환자임'

In [0]:
ner.extraction_to_print(spacing('상기 환자 진단 하 2023.4.12 입원하여 동년동월 13일 복강경하 좌측난소 낭종절제술, 자궁근종절제술 시행받고 동년동월 16일퇴원한 환자임'))

입원일 : []
퇴원일 : []
수술일 : ['2023.04.12']
수술명 : ['복강경하 좌측난소 낭종절제술', '자궁근종절제술']


In [0]:
ner.extraction_to_print(spacing('상기인 2023년 8월 22일 본원에서 시행한 대장내시경상 용종이 관찰되어, 올가미 용종절제술을 이용하여 제거하였습니다. 조직 검사 결과 " 저도 이형성을 동반한 관상 선종"으로 확인되었습니다 향후 대장내시경을 이용한 경과 관찰을 요합니다'))

입원일 : []
퇴원일 : []
수술일 : ['2023.08.22']
수술명 : ['올가미 용종절제술']


In [0]:
ner(spacing('상기 환자분 상기상병명으로 2021.3.16일 제왕절개술후 3.16-3.20까지 입원치료 받고 2022.8.30일 제왕절개술후 8.30-9.5까지 입원치료함.'))

Out[42]: [{'word': '상기', 'score': 0.9327638149261475, 'entity': 'O'},
 {'word': '환자분', 'score': 0.9327996969223022, 'entity': 'O'},
 {'word': '상기상병명', 'score': 0.9327890872955322, 'entity': 'O'},
 {'word': '2021.03.16', 'score': 0.7365450859069824, 'entity': 'DATE_S'},
 {'word': '일', 'score': 0.9322234988212585, 'entity': 'O'},
 {'word': '제왕절개술후', 'score': 0.7775954008102417, 'entity': 'SURG'},
 {'word': '2021.03.16', 'score': 0.5225392580032349, 'entity': 'DATE_S'},
 {'word': '-', 'score': 0.9321548342704773, 'entity': 'O'},
 {'word': '2021.03.20', 'score': 0.4764212667942047, 'entity': 'DATE_R'},
 {'word': '입원치료', 'score': 0.8780016303062439, 'entity': 'O'},
 {'word': '받고', 'score': 0.9323380589485168, 'entity': 'O'},
 {'word': '2022.08.30', 'score': 0.4265369474887848, 'entity': 'DATE_H'},
 {'word': '일', 'score': 0.932198166847229, 'entity': 'O'},
 {'word': '제왕절개술후', 'score': 0.7798081040382385, 'entity': 'SURG'},
 {'word': '2022.08.30', 'score': 0.47379228472709656, 'entity': 'DATE

In [0]:
ner.extraction_to_print(spacing('[입원일자]: 2023-05-04 ~ 2023-05-07  [수술일자] : 2023-05-04 / [수술명] : 복강경하 자궁근종 절제술 + 자국내막 소파술   [산부인과/외래] 2023-05-12  [산부인과/외래] 2023-04-28   [산부인과/외래] 2023-04-26'))

입원일 : ['2023.05.04', '2023.04.28']
퇴원일 : ['2023.05.07', '2023.05.12', '2023.04.26']
수술일 : ['2023.05.04']
수술명 : ['복강경하 자궁근종 절제술', '자국내막 소파술']
